In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('archive (6).zip','r')
zip_ref.extractall()
zip_ref.close()
data_dir='archive (6)/dataset-resized'

In [ ]:
train_ds=tf.keras.utils.image_dataset_from_directory(data_dir,
                                                     validation_split=0.2,
                                                     label_mode='int',
                                                     subset='training',
                                                     seed=1227,
                                                     image_size=(224,224),
                                                     batch_size=32
                                                     )

Found 1987 files belonging to 4 classes.
Using 1590 files for training.


In [ ]:
valid_ds=tf.keras.utils.image_dataset_from_directory(data_dir,
                                                     validation_split=0.2,
                                                     label_mode='int',
                                                     subset='validation',
                                                     seed=1227,
                                                     image_size=(224,224),
                                                     batch_size=32
)

Found 1987 files belonging to 4 classes.
Using 397 files for validation.


In [ ]:
class_names=train_ds.class_names
print(class_names)
num_classes=len(class_names)
print(num_classes)

['glass', 'metal', 'paper', 'plastic']
4


In [ ]:
val_batches=tf.data.experimental.cardinality(valid_ds)
test_ds=valid_ds.take(val_batches//5)
valid_ds=valid_ds.skip(val_batches//5)

In [ ]:
print('Number of validation batches: %d' %tf.data.experimental.cardinality(valid_ds))
print('Number of test batches: %d' %tf.data.experimental.cardinality(test_ds))


AUTOTUNE=tf.data.AUTOTUNE
train_ds=train_ds.prefetch(buffer_size=AUTOTUNE)
valid_ds=valid_ds.prefetch(buffer_size=AUTOTUNE)
test_ds=test_ds.prefetch(buffer_size=AUTOTUNE)

Number of validation batches: 11
Number of test batches: 2


In [ ]:
# use data augmentation
data_augmentation=tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2)
])

In [ ]:
inputs=tf.keras.Input(shape=(224,224,3))

# choose the model to use
base_model=tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras import layers as tkl
base_model.trainable=False
inputs=tf.keras.Input(shape=(224,224,3))
x=data_augmentation(inputs)
x=base_model(x, training=False)
x=tkl.GlobalAveragePooling2D()(x)
x=tkl.BatchNormalization()(x)
x=tkl.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x=tkl.Dropout(0.5)(x)
x=tkl.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x=tkl.Dropout(0.5)(x)
outputs=tkl.Dense(num_classes, activation='softmax')(x)

In [ ]:
model=tf.keras.Model(inputs,outputs)
base_learning_rate=0.001

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
initial_epochs=60
callback_list=[ModelCheckpoint(filepath='model.keras',
                               monitor='val_accuracy',
                               verbose=1,
                               save_best_only=True,
                               save_weights_only=False,
                               mode='max')]
history=model.fit(train_ds,
                  epochs=initial_epochs,
                  validation_data=valid_ds,
                  callbacks=(callback_list))

Epoch 1/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.5067 - loss: 13.8288
Epoch 1: val_accuracy improved from -inf to 0.78378, saving model to model.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 26s 318ms/step - accuracy: 0.5090 - loss: 13.8056 - val_accuracy: 0.7838 - val_loss: 10.2672
Epoch 2/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.7394 - loss: 9.5336
Epoch 2: val_accuracy improved from 0.78378 to 0.85886, saving model to model.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 222ms/step - accuracy: 0.7398 - loss: 9.5185 - val_accuracy: 0.8589 - val_loss: 7.0955
Epoch 3/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.8297 - loss: 6.5279
Epoch 3: val_accuracy improved from 0.85886 to 0.88589, saving model to model.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 223ms/step - accuracy: 0.8297 - loss: 6.5173 - val_accuracy: 0.8859 - val_loss: 4.8274
Epoch 4/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.8523 - loss: 4.5009
Epoch 4: val_accuracy did not improve from 0.88589

In [ ]:
print('NUmber of layers in the base model:', len(base_model.layers))

NUmber of layers in the base model: 22


In [ ]:
for layer in base_model.layers:
  layer.trainable=True

In [ ]:
base_model.trainable=True
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
total_epochs=120
history_fine=model.fit(train_ds,
                       epochs=total_epochs,
                       initial_epoch=history.epoch[-1],
                       validation_data=valid_ds,
                       callbacks=(callback_list))

Epoch 60/120
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 600ms/step - accuracy: 0.6893 - loss: 1.2134
Epoch 60: val_accuracy did not improve from 0.90390
50/50 ━━━━━━━━━━━━━━━━━━━━ 44s 658ms/step - accuracy: 0.6887 - loss: 1.2148 - val_accuracy: 0.6216 - val_loss: 2.2715
Epoch 61/120
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 525ms/step - accuracy: 0.7313 - loss: 1.0946
Epoch 61: val_accuracy did not improve from 0.90390
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 565ms/step - accuracy: 0.7313 - loss: 1.0946 - val_accuracy: 0.5646 - val_loss: 2.1558
Epoch 62/120
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 536ms/step - accuracy: 0.7437 - loss: 1.0542
Epoch 62: val_accuracy did not improve from 0.90390
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 589ms/step - accuracy: 0.7432 - loss: 1.0552 - val_accuracy: 0.5886 - val_loss: 2.4960
Epoch 63/120
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 530ms/step - accuracy: 0.7596 - loss: 1.0044
Epoch 63: val_accuracy did not improve from 0.90390
50/50 ━━━━━━━━━━━━━━━━━━━━ 40s 570ms/step - accuracy: 0.7601 - loss: 1.0033 - val_accu

In [ ]:
accuracy,loss=model.evaluate(test_ds)
print(f'loss: {loss} , accuracy: {accuracy}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.9688 - loss: 0.2805
loss: 0.96875 , accuracy: 0.26552215218544006


In [ ]:
model.save('waste_class_model.keras')

In [ ]:
!python -m pip install tensorflow==2.12.0 wandb edgeimpulse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440

In [ ]:
import tensorflow as tf
from tensorflow import keras
import edgeimpulse as ei
ei.API_KEY='ei_c7a4de1d4f37f791a32dcf76b38c5cd8cb1ec1605bc4167cc47412461a0bdc30'

ModuleNotFoundError: No module named 'edgeimpulse'

In [ ]:
labels=['0','1','2','3']
deploy_filename='my_model_cpp.zip'
ei.model.list_profile_devices()

['alif-he',
 'alif-hp',
 'ambiq-apollo4',
 'arduino-nano-33-ble',
 'arduino-nicla-vision',
 'arduino-nicla-vision-m4',
 'portenta-h7',
 'brainchip-akd1000',
 'brickml',
 'cortex-m4f-80mhz',
 'cortex-m7-216mhz',
 'espressif-esp32',
 'himax-we-i',
 'himax-wiseeye2',
 'himax-wiseeye2-ethos',
 'infineon-cy8ckit-062s2',
 'infineon-cy8ckit-062-ble',
 'mbp-16-2020',
 'memryx-mx3',
 'microchip-sama7g54',
 'nordic-nrf52840-dk',
 'nordic-nrf5340-dk',
 'nordic-nrf9160-dk',
 'jetson-nano',
 'jetson-orin-nx',
 'jetson-orin-nano',
 'openmv-h7p',
 'particle-boron',
 'particle-p2',
 'raspberry-pi-4',
 'raspberry-pi-5',
 'raspberry-pi-rp2040',
 'renesas-ck-ra6m5',
 'renesas-ek-ra8d1',
 'renesas-rzg2l',
 'renesas-rzv2l-cpu',
 'renesas-rzv2l',
 'st-iot-discovery-kit',
 'seeed-sense-cap',
 'wio-terminal',
 'seeed-vision-ai',
 'silabs-xg24',
 'silabs-thunderboard-sense-2',
 'sony-spresense',
 'synaptics-ka10000',
 'ti-am62a',
 'ti-am68a',
 'ti-launchxl',
 'ti-tda4vm',
 'neox']

In [ ]:
'''from edgeimpulse import Client

# Initialize client
client = Client(api_key="ei_c7a4de1d4f37f791a32dcf76b38c5cd8cb1ec1605bc4167cc47412461a0bdc30")

# Upload the model to Edge Impulse
client.upload_pretrained_model(
    project_id='522189',
    model_path=r"C:\Users\PASCAL\Downloads\waste_class_model.keras",
    model_type="keras"
)'''


ImportError: cannot import name 'Client' from 'edgeimpulse' (/usr/local/lib/python3.10/dist-packages/edgeimpulse/__init__.py)

In [ ]:
try:
  profile=ei.model.profile(model='/content/waste_class_model.keras',
                           device='raspberry-pi-4')
  print(profile.summary())
except Exception as e:
  print(f'could not profile: {e}')

could not profile: No uploaded model yet


In [ ]:
pip show tensorflow keras

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras
---
Name: keras
Version: 2.12.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: tensorflow


In [ ]:
import tensorflow as tf


In [ ]:
keras_model=tf.keras.models.load_model('/content/waste_class_model.keras')
keras_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg19 (Functional)                   │ (None, 7, 7, 512)           │      20,024,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 63,233,742 (241.22 MB)

 Trainable params: 21,077,572 (80.40 MB)

 Non-trainable params: 1,024 (4.00 KB)

 Optimizer params: 42,155,146 (160.81 MB)

In [ ]:
converter=tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflitemodel=converter.convert()

Saved artifact at '/tmp/tmp6lij9er5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_6')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  140371025854160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981931488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981932896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981936416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981934304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981939760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981937120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981942224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981938528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140370981940992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1403709819438

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflitemodel)